In [39]:
from datetime import datetime, timedelta

import pandas as pd

In [76]:
train_v1 = pd.read_csv("../data/raw/v1/train.csv", encoding="cp949")
train_v1 = train_v1[(train_v1["승차X좌표"] != 0) & (train_v1["승차Y좌표"] != 0)]
train_v1['승차시간'] = pd.to_datetime(train_v1['승차시간'])
train_v1.shape

(1082657, 5)

In [77]:
train_v1.head()

,차량번호,승차시간,승차X좌표,승차Y좌표,요일
44,sqonam,2023-04-01 05:15:37,127.377749,36.350133,Saturday
45,sqonam,2023-04-01 05:42:47,127.358606,36.353338,Saturday
46,sqonam,2023-04-01 06:02:58,127.380240,36.365618,Saturday
47,sqonam,2023-04-01 06:15:05,127.426666,36.351172,Saturday
48,sqonam,2023-04-01 06:28:36,127.408217,36.324442,Saturday


In [85]:
# 승/하차시간 열을 정제
def clean_drop_off_time(value):
    if isinstance(value, datetime):
        return value
    if isinstance(value, int):
        value = str(value)
    if isinstance(value, str):
        # 'YYYYMMDDHHMMSS' 형식인 경우
        if len(value) == 14 and value.isdigit():
            return datetime.strptime(value, '%Y%m%d%H%M%S').strftime('%Y-%m-%d %H:%M:%S')
        # 이미 'YYYY-MM-DD HH:MM:SS' 형식인 경우
        elif len(value) == 19 and value[4] == '-' and value[13] == ':':
            return value
    # 다른 모든 경우 (NaT 포함)
    return pd.NaT

train_v3 = pd.read_csv("../data/raw/v3/train.csv", encoding="cp949")
train_v3 = train_v3[(train_v3["승차X좌표"] != 0) & (train_v3["승차Y좌표"] != 0)]
train_v3['승차시간'] = train_v3['승차시간'].apply(clean_drop_off_time)
train_v3['하차시간'] = train_v3['하차시간'].apply(clean_drop_off_time)
train_v3['승차시간'] = pd.to_datetime(train_v3['승차시간'])
train_v3['하차시간'] = pd.to_datetime(train_v3['하차시간'])
train_v3['승차시간'].where(train_v3['승차시간'].dt.year != 1970, pd.NaT, inplace=True)
train_v3['하차시간'].where(train_v3['하차시간'].dt.year != 1970, pd.NaT, inplace=True)
train_v3.shape

/var/folders/yk/1k0z841d31x0hr5nbkdxbsxm0000gn/T/ipykernel_3531/502623032.py:17: DtypeWarning: Columns (1,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  train_v3 = pd.read_csv("../data/raw/v3/train.csv", encoding="cp949")


(1151410, 11)

In [87]:
train_v3.head()

,차량이름,승차시간,승차요일,승차X좌표,승차Y좌표,하차시간,하차X좌표,하차Y좌표,승차거리(m),할증여부,요금
0,xqbxch,NaT,Tuesday,127.367251,36.349866,2023-06-20 11:08:21,127.367251,36.349866,0.00,미할증,3300
1,xqbxch,NaT,Tuesday,127.367263,36.349684,2023-06-20 11:11:00,127.367263,36.349684,0.00,미할증,3300
3,xqbxch,NaT,Tuesday,127.340568,36.298811,2023-06-20 22:27:20,127.350915,36.305854,1518.13,미할증,3300
4,xqbxch,NaT,Tuesday,127.351858,36.299839,2023-06-20 22:47:15,127.374955,36.345087,7045.65,미할증,8000
5,xqbxch,NaT,Tuesday,127.371055,36.346926,2023-06-20 22:54:43,127.376224,36.350242,738.03,미할증,3300


In [132]:
# 데이터 전처리: 컬럼명 통일 및 데이터 타입 변환
train_v1 = train_v1.rename(columns={'차량번호': '차량이름', '요일': '승차요일'})

# 병합을 위한 키 생성
train_v1['merge_key'] = train_v1['차량이름'] + train_v1['승차요일'] + train_v1['승차X좌표'].round(6).astype(str) + train_v1['승차Y좌표'].round(6).astype(str)
train_v3['merge_key'] = train_v3['차량이름'] + train_v3['승차요일'] + train_v3['승차X좌표'].round(6).astype(str) + train_v3['승차Y좌표'].round(6).astype(str)

# 데이터 병합
train = pd.merge(train_v3, train_v1[['merge_key', '승차시간']], on='merge_key', how='left', suffixes=('', '_train_v1'))

# 조건에 맞는 경우 승차시간 업데이트
train.loc[train['승차시간'].isna(), '승차시간'] = train.loc[train['승차시간'].isna(), '승차시간_train_v1']

# 불필요한 컬럼 제거
train = train.drop(['merge_key', '승차시간_train_v1'], axis=1)

In [133]:
train.dropna(inplace=True)

In [134]:
train.drop_duplicates(subset=['차량이름', '승차시간'], keep=False, inplace=True)

In [143]:
train = train[train['승차거리(m)'] != 0]

In [144]:
train.shape
(1151410, 11)

(890645, 11)

In [145]:
train.head()

,차량이름,승차시간,승차요일,승차X좌표,승차Y좌표,하차시간,하차X좌표,하차Y좌표,승차거리(m),할증여부,요금
0,acfgkx,2023-04-01 00:01:00,Saturday,127.377074,36.351814,2023-04-01 00:09:04,127.344545,36.357077,4427.0,할증,6600
1,acfgkx,2023-04-01 00:09:00,Saturday,127.344478,36.357087,2023-04-01 00:14:48,127.364499,36.354269,2827.0,할증,5160
3,acfgkx,2023-04-01 00:27:00,Saturday,127.392290,36.375748,2023-04-01 00:34:52,127.344714,36.354130,5813.0,할증,7920
4,acfgkx,2023-04-01 00:37:00,Saturday,127.351769,36.358168,2023-04-01 00:45:50,127.349764,36.392184,6436.0,할증,8520
5,acfgkx,2023-04-01 00:55:00,Saturday,127.359606,36.363154,2023-04-01 01:06:12,127.308421,36.343433,6427.0,할증,8760
